In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
import altair as alt

In [2]:
df = pd.read_csv('vehicles_us.csv')

In [3]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [4]:
display(df.dtypes)

price             int64
model_year      float64
model            object
condition        object
cylinders       float64
fuel             object
odometer        float64
transmission     object
type             object
paint_color      object
is_4wd          float64
date_posted      object
days_listed       int64
dtype: object

In [5]:
nan_rows =df[df.isna().any(axis=1)]
display(nan_rows)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,nissan maxima,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37
51521,2700,2002.0,honda civic,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22
51522,3950,2009.0,hyundai sonata,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32
51523,7455,2013.0,toyota corolla,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71


In [6]:
# droped all NAN except for the color column which is replaced with other. The other columns with NAN were dropped because they have incomplete details that would hinder a persons search. 
#If the details are needed we would v to obtain prior to adding to analysis.
df['paint_color'] =df['paint_color'].fillna('other')
df=df.dropna()
display(df)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,other,1.0,2018-06-23,19
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,other,1.0,2018-07-16,19
10,19500,2011.0,chevrolet silverado 1500,excellent,8.0,gas,128413.0,automatic,pickup,black,1.0,2018-09-17,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51500,6995,2012.0,buick enclave,excellent,6.0,gas,157000.0,automatic,SUV,white,1.0,2018-08-04,17
51506,28900,2017.0,ford f150 supercrew cab xlt,good,6.0,gas,49225.0,automatic,pickup,other,1.0,2019-04-11,25
51509,5980,2010.0,subaru impreza,excellent,4.0,gas,121778.0,manual,hatchback,white,1.0,2019-01-10,8
51513,17999,2014.0,ram 1500,like new,8.0,gas,154000.0,automatic,pickup,white,1.0,2018-06-21,11


In [7]:
#clean  up model year and the odometer column by converting to int.64
df['model_year'] =df['model_year'].astype(int)
df['odometer'] =df['odometer'].astype(int)
display(df.dtypes)

price             int64
model_year        int64
model            object
condition        object
cylinders       float64
fuel             object
odometer          int64
transmission     object
type             object
paint_color      object
is_4wd          float64
date_posted      object
days_listed       int64
dtype: object

In [8]:
# split model column to create the first part of the string as the manufacturer and keep the sec part of the string as model.
df[['manufacturer', 'model']] = df['model'].str.split(' ', n=1, expand=True)
df.head()


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
0,9400,2011,x5,good,6.0,gas,145000,automatic,SUV,other,1.0,2018-06-23,19,bmw
5,14990,2014,300,excellent,6.0,gas,57954,automatic,sedan,black,1.0,2018-06-20,15,chrysler
7,15990,2013,pilot,excellent,6.0,gas,109473,automatic,SUV,black,1.0,2019-01-07,68,honda
8,11500,2012,sorento,excellent,4.0,gas,104174,automatic,SUV,other,1.0,2018-07-16,19,kia
10,19500,2011,silverado 1500,excellent,8.0,gas,128413,automatic,pickup,black,1.0,2018-09-17,38,chevrolet


In [9]:
#move the manufacturer column to be close to the front of model to keep unison.
desired_order =  ['price', 'model_year', 'manufacturer', 'model', 'date_posted', 'days_listed', 'condition', 'cylinders', 'fuel', 'odometer', 'transmission', 'type', 'paint_color', 'is_4wd']
df = df[desired_order]
df.head()

,price,model_year,manufacturer,model,date_posted,days_listed,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd
0,9400,2011,bmw,x5,2018-06-23,19,good,6.0,gas,145000,automatic,SUV,other,1.0
5,14990,2014,chrysler,300,2018-06-20,15,excellent,6.0,gas,57954,automatic,sedan,black,1.0
7,15990,2013,honda,pilot,2019-01-07,68,excellent,6.0,gas,109473,automatic,SUV,black,1.0
8,11500,2012,kia,sorento,2018-07-16,19,excellent,4.0,gas,104174,automatic,SUV,other,1.0
10,19500,2011,chevrolet,silverado 1500,2018-09-17,38,excellent,8.0,gas,128413,automatic,pickup,black,1.0


In [10]:
#convert date_posted column to datetime format
df['date_posted'] = pd.to_datetime(df['date_posted'])
display(df.dtypes)

price                    int64
model_year               int64
manufacturer            object
model                   object
date_posted     datetime64[ns]
days_listed              int64
condition               object
cylinders              float64
fuel                    object
odometer                 int64
transmission            object
type                    object
paint_color             object
is_4wd                 float64
dtype: object

In [11]:
#the oldest listing in the dataset:
df['days_listed'].max()

np.int64(263)

In [12]:
#The newest listing ion the dataset:
df['days_listed'].min()

np.int64(0)

In [13]:
#Highest price listed for a vehicle
df['price'].max()

np.int64(375000)

In [14]:
#lowest to highest milage listed:
lowest_mls, highest_mls = int(df['odometer'].min()), int(df['odometer'].max())

In [15]:
print(highest_mls)

866000


In [16]:
#How old is the car?
df['age'] = 2024 -df['model_year']
df.head()

,price,model_year,manufacturer,model,date_posted,days_listed,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,age
0,9400,2011,bmw,x5,2018-06-23,19,good,6.0,gas,145000,automatic,SUV,other,1.0,13
5,14990,2014,chrysler,300,2018-06-20,15,excellent,6.0,gas,57954,automatic,sedan,black,1.0,10
7,15990,2013,honda,pilot,2019-01-07,68,excellent,6.0,gas,109473,automatic,SUV,black,1.0,11
8,11500,2012,kia,sorento,2018-07-16,19,excellent,4.0,gas,104174,automatic,SUV,other,1.0,12
10,19500,2011,chevrolet,silverado 1500,2018-09-17,38,excellent,8.0,gas,128413,automatic,pickup,black,1.0,13


In [17]:
df.to_csv('preprocessed_data_vehicles_us.csv')
